In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install tensorflow

In [3]:
import numpy as np
from tensorflow.keras.utils import to_categorical

# ✅ Reload dataset from Google Drive
save_path = "/content/drive/MyDrive/Indian_Musician_Project/preprocessed_data"

X_train = np.load(f"{save_path}/X_train.npy")
y_train = np.load(f"{save_path}/y_train.npy")
X_val = np.load(f"{save_path}/X_val.npy")
y_val = np.load(f"{save_path}/y_val.npy")

# ✅ Print dataset shape
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")

# ✅ Fix labels
if len(y_train.shape) == 1 or y_train.shape[1] != 11:
    y_train = to_categorical(y_train, num_classes=11)
    y_val = to_categorical(y_val, num_classes=11)

# ✅ Print to verify
print(f"✅ Labels Fixed! New y_train shape: {y_train.shape}")

X_train shape: (536, 128, 128, 3), y_train shape: (536,)
✅ Labels Fixed! New y_train shape: (536, 11)


In [4]:
X_train = X_train / 255.0
X_val = X_val / 255.0

print("✅ Data Normalized!")

✅ Data Normalized!


In [11]:
from tensorflow.keras.utils import to_categorical

# Convert integer labels to one-hot encoding
y_test = to_categorical(y_test, num_classes=11)

print("✅ Fixed y_test shape:", y_test.shape)  # Should be (68, 11)

✅ Fixed y_test shape: (68, 11)


In [16]:
# ✅ If y_train and y_val are (536, 11, 11), keep only the first row
if len(y_train.shape) == 3:
    y_train = y_train[:, 0, :]
if len(y_val.shape) == 3:
    y_val = y_val[:, 0, :]

# ✅ Print final shapes
print("✅ Fixed y_train shape:", y_train.shape)  # Should be (536, 11)
print("✅ Fixed y_val shape:", y_val.shape)      # Should be (67, 11)
print("✅ Fixed y_test shape:", y_test.shape)    # Should already be (68, 11)

✅ Fixed y_train shape: (536, 11)
✅ Fixed y_val shape: (67, 11)
✅ Fixed y_test shape: (68, 11)


In [17]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# ✅ Define Image Augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True
)

# ✅ Apply Augmentation to Training Data
datagen.fit(X_train)

In [18]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

# ✅ Reduce Learning Rate if Validation Accuracy Stagnates
lr_scheduler = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=3,
    min_lr=1e-6,
    verbose=1
)

In [19]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor="val_loss",
    patience=5,  # Stops training if no improvement for 5 epochs
    restore_best_weights=True,
    verbose=1
)

In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, Dense, Dropout

from tensorflow.keras.layers import Input

model = Sequential([
    Input(shape=(128, 128, 3)),  # ✅ Define input separately
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Conv2D(256, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Conv2D(512, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    GlobalAveragePooling2D(),

    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),

    Dense(11, activation='softmax')  # 11 classes for musicians
])


# ✅ Compile Model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# ✅ Model Summary
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)                    │ (None, 128, 128, 128)       │           3,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 128, 128, 128)       │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 64, 64, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 64, 64, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_5                │ (None, 64, 64, 256)         │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 32, 32, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 32, 32, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_6                │ (None, 32, 32, 512)         │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 16, 16, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_1           │ (None, 512)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_7                │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 11)                  │           2,827 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,617,675 (6.17 MB)

 Trainable params: 1,615,371 (6.16 MB)

 Non-trainable params: 2,304 (9.00 KB)

In [24]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=30,  # Increase if needed
    batch_size=32
)

Epoch 1/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 32s 1s/step - accuracy: 0.1200 - loss: 3.4726 - val_accuracy: 0.9104 - val_loss: 2.0366
Epoch 2/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 13s 93ms/step - accuracy: 0.2322 - loss: 2.7131 - val_accuracy: 0.9104 - val_loss: 1.7092
Epoch 3/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 2s 92ms/step - accuracy: 0.3002 - loss: 2.4682 - val_accuracy: 0.9104 - val_loss: 1.5814
Epoch 4/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 2s 98ms/step - accuracy: 0.4274 - loss: 2.0499 - val_accuracy: 0.9104 - val_loss: 1.4042
Epoch 5/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 2s 99ms/step - accuracy: 0.4728 - loss: 1.8899 - val_accuracy: 0.9104 - val_loss: 1.0701
Epoch 6/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 2s 92ms/step - accuracy: 0.5436 - loss: 1.6743 - val_accuracy: 0.9104 - val_loss: 0.9860
Epoch 7/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 92ms/step - accuracy: 0.6287 - loss: 1.3281 - val_accuracy: 0.9104 - val_loss: 0.4531
Epoch 8/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 93ms/step - accuracy: 0.7355 - loss: 1.1150 - val_accuracy: 0.9104 - v

In [25]:
#Save the Trained Model
model.save('/content/drive/MyDrive/musician_recognition_model1.h5')


In [30]:
import numpy as np
from tensorflow.keras.utils import to_categorical

# ✅ Define the dataset path
save_path = "/content/drive/MyDrive/Indian_Musician_Project/preprocessed_data"

# ✅ Load dataset from Google Drive
X_train = np.load(f"{save_path}/X_train.npy")
y_train = np.load(f"{save_path}/y_train.npy")
X_val = np.load(f"{save_path}/X_val.npy")
y_val = np.load(f"{save_path}/y_val.npy")
X_test = np.load(f"{save_path}/X_test.npy")
y_test = np.load(f"{save_path}/y_test.npy")

# ✅ Print dataset shapes
print(f"✅ X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"✅ X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")
print(f"✅ X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

# ✅ Fix labels if they are not one-hot encoded
if len(y_train.shape) == 1 or y_train.shape[1] != 11:
    y_train = to_categorical(y_train, num_classes=11)
    y_val = to_categorical(y_val, num_classes=11)
    y_test = to_categorical(y_test, num_classes=11)

# ✅ Print to verify
print(f"✅ Labels Fixed! New y_train shape: {y_train.shape}")
print(f"✅ Labels Fixed! New y_val shape: {y_val.shape}")
print(f"✅ Labels Fixed! New y_test shape: {y_test.shape}")

# ✅ Evaluate Model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"🎯 Test Accuracy: {test_acc:.2%}, Test Loss: {test_loss:.4f}")

✅ X_train shape: (536, 128, 128, 3), y_train shape: (536,)
✅ X_val shape: (67, 128, 128, 3), y_val shape: (67,)
✅ X_test shape: (68, 128, 128, 3), y_test shape: (68,)
✅ Labels Fixed! New y_train shape: (536, 11)
✅ Labels Fixed! New y_val shape: (67, 11)
✅ Labels Fixed! New y_test shape: (68, 11)
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 833ms/step - accuracy: 0.0676 - loss: 1326.8658
🎯 Test Accuracy: 8.82%, Test Loss: 1274.0885


In [31]:
import numpy as np

# Convert one-hot labels back to class indices
y_train_classes = np.argmax(y_train, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

# Check distribution
unique, counts = np.unique(y_train_classes, return_counts=True)
print("Class distribution in y_train:", dict(zip(unique, counts)))

unique, counts = np.unique(y_test_classes, return_counts=True)
print("Class distribution in y_test:", dict(zip(unique, counts)))

Class distribution in y_train: {np.int64(0): np.int64(49), np.int64(1): np.int64(49), np.int64(2): np.int64(49), np.int64(3): np.int64(48), np.int64(4): np.int64(49), np.int64(5): np.int64(49), np.int64(6): np.int64(48), np.int64(7): np.int64(48), np.int64(8): np.int64(49), np.int64(9): np.int64(49), np.int64(10): np.int64(49)}
Class distribution in y_test: {np.int64(0): np.int64(6), np.int64(1): np.int64(6), np.int64(2): np.int64(6), np.int64(3): np.int64(6), np.int64(4): np.int64(6), np.int64(5): np.int64(6), np.int64(6): np.int64(7), np.int64(7): np.int64(7), np.int64(8): np.int64(6), np.int64(9): np.int64(6), np.int64(10): np.int64(6)}


In [32]:
from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight('balanced', classes=np.unique(y_train_classes), y=y_train_classes)
class_weight_dict = dict(enumerate(class_weights))

# Train with class weights
history = model.fit(X_train, y_train, validation_data=(X_val, y_val),
                    epochs=30, batch_size=32, class_weight=class_weight_dict)

Epoch 1/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 9s 331ms/step - accuracy: 0.0765 - loss: 6.7918 - val_accuracy: 0.0896 - val_loss: 382.8137
Epoch 2/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 2s 95ms/step - accuracy: 0.1388 - loss: 3.6454 - val_accuracy: 0.0896 - val_loss: 212.7104
Epoch 3/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 2s 100ms/step - accuracy: 0.1816 - loss: 2.8490 - val_accuracy: 0.0896 - val_loss: 114.7397
Epoch 4/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 2s 94ms/step - accuracy: 0.1955 - loss: 2.4301 - val_accuracy: 0.0896 - val_loss: 40.9513
Epoch 5/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 2s 99ms/step - accuracy: 0.2541 - loss: 2.4404 - val_accuracy: 0.0896 - val_loss: 29.5565
Epoch 6/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 102ms/step - accuracy: 0.2096 - loss: 2.2245 - val_accuracy: 0.0896 - val_loss: 13.9197
Epoch 7/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 2s 96ms/step - accuracy: 0.2533 - loss: 2.1265 - val_accuracy: 0.0896 - val_loss: 17.2279
Epoch 8/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 99ms/step - accuracy: 0.2410 - loss: 2.2645 - val_accurac

In [33]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model

# Load MobileNetV2 (without top layers)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
base_model.trainable = False  # Freeze layers

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(11, activation='softmax')(x)

# Create final model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=30, batch_size=32)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 19s 586ms/step - accuracy: 0.0965 - loss: 3.0182 - val_accuracy: 0.0896 - val_loss: 2.3043
Epoch 2/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 7s 22ms/step - accuracy: 0.2161 - loss: 2.1957 - val_accuracy: 0.1642 - val_loss: 2.1853
Epoch 3/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.2961 - loss: 1.9760 - val_accuracy: 0.2388 - val_loss: 2.1477
Epoch 4/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.3192 - loss: 1.8989 - val_accuracy: 0.2687 - val_loss: 2.1606
Epoch 5/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.3762 - loss: 1.6986 - val_accuracy: 0.2537 - val_loss: 2.1922
Epoch 6/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.3847 - loss: 1.6834 - val_accuracy: 0.2239 - val_loss: 2.1497
Epoch 7/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.4006 - loss: 1.6553 - val_accuracy: 0.2537 - val_loss: 2.0467
Epoch 8/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accurac

In [34]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Define new optimizer with lower LR
optimizer = Adam(learning_rate=0.0001)  # Reduce from 0.001 to 0.0001

# Compile model with new optimizer
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Reduce Learning Rate if no improvement
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)

# Retrain the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=30,
    batch_size=32,
    callbacks=[lr_scheduler]  # Add learning rate scheduler
)

Epoch 1/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 18s 663ms/step - accuracy: 0.8765 - loss: 0.4735 - val_accuracy: 0.2836 - val_loss: 2.5429 - learning_rate: 1.0000e-04
Epoch 2/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 8s 22ms/step - accuracy: 0.9134 - loss: 0.4190 - val_accuracy: 0.2687 - val_loss: 2.5477 - learning_rate: 1.0000e-04
Epoch 3/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.9083 - loss: 0.3994 - val_accuracy: 0.2687 - val_loss: 2.5370 - learning_rate: 1.0000e-04
Epoch 4/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.9222 - loss: 0.3780 - val_accuracy: 0.2687 - val_loss: 2.5365 - learning_rate: 1.0000e-04
Epoch 5/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.9277 - loss: 0.3633 - val_accuracy: 0.2687 - val_loss: 2.5463 - learning_rate: 1.0000e-04
Epoch 6/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.9020 - loss: 0.3892 - val_accuracy: 0.2836 - val_loss: 2.5552 - learning_rate: 1.0000e-04
Epoch 7/30
14/17 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.936

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(np.argmax(y_train, axis=1)),
    y=np.argmax(y_train, axis=1)
)

class_weight_dict = dict(enumerate(class_weights))
print(f"✅ Computed Class Weights: {class_weight_dict}")

✅ Computed Class Weights: {0: np.float64(0.5503080082135524), 1: np.float64(5.469387755102041)}


In [ ]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=30,
    batch_size=32,
    class_weight=class_weight_dict  # 🛠️ Fix imbalance!
)

Epoch 1/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 8s 256ms/step - accuracy: 0.7035 - loss: 0.9296 - val_accuracy: 0.2388 - val_loss: 1.0897
Epoch 2/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 2s 92ms/step - accuracy: 0.7431 - loss: 0.9362 - val_accuracy: 0.1045 - val_loss: 1.7815
Epoch 3/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 2s 90ms/step - accuracy: 0.7073 - loss: 0.9477 - val_accuracy: 0.1045 - val_loss: 2.4057
Epoch 4/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 88ms/step - accuracy: 0.6655 - loss: 0.7841 - val_accuracy: 0.1045 - val_loss: 2.3134
Epoch 5/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 88ms/step - accuracy: 0.6504 - loss: 0.8231 - val_accuracy: 0.1194 - val_loss: 2.4105
Epoch 6/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 93ms/step - accuracy: 0.6930 - loss: 0.6634 - val_accuracy: 0.1343 - val_loss: 2.4388
Epoch 7/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 2s 90ms/step - accuracy: 0.6304 - loss: 0.8649 - val_accuracy: 0.1493 - val_loss: 2.1873
Epoch 8/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 2s 95ms/step - accuracy: 0.6750 - loss: 0.7553 - val_accuracy: 0.1642 - 

In [ ]:
from tensorflow.keras.layers import Dropout
from tensorflow.keras.regularizers import l2

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation="relu", input_shape=(128, 128, 3)),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(64, (3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(128, (3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Flatten(),

    # ✅ L2 Regularization & Dropout
    tf.keras.layers.Dense(128, activation="relu", kernel_regularizer=l2(0.01)),
    Dropout(0.4),  # 40% Dropout to prevent overfitting

    tf.keras.layers.Dense(11, activation="softmax")
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

history = model.fit(
    datagen.flow(X_train, y_train, batch_size=32),
    validation_data=(X_val, y_val),
    epochs=30,
    class_weight=class_weight_dict
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 12s 490ms/step - accuracy: 0.6383 - loss: 3.7279 - val_accuracy: 0.9104 - val_loss: 1.1638
Epoch 2/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 12s 115ms/step - accuracy: 0.4976 - loss: 1.3083 - val_accuracy: 0.0896 - val_loss: 1.3103
Epoch 3/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 2s 114ms/step - accuracy: 0.2968 - loss: 1.1138 - val_accuracy: 0.9104 - val_loss: 0.7977
Epoch 4/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 2s 119ms/step - accuracy: 0.5510 - loss: 0.9853 - val_accuracy: 0.0896 - val_loss: 0.8480
Epoch 5/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 150ms/step - accuracy: 0.3527 - loss: 0.8983 - val_accuracy: 0.9104 - val_loss: 0.7057
Epoch 6/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 5s 115ms/step - accuracy: 0.6053 - loss: 0.8205 - val_accuracy: 0.9104 - val_loss: 0.7632
Epoch 7/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 2s 114ms/step - accuracy: 0.5362 - loss: 0.8217 - val_accuracy: 0.9104 - val_loss: 0.6591
Epoch 8/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 2s 117ms/step - accuracy: 0.5330 - loss: 0.8527 - val_accuracy: 

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"🎯 New Test Accuracy: {test_acc:.2%}, New Test Loss: {test_loss:.4f}")

3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 375ms/step - accuracy: 0.9324 - loss: 0.6983
🎯 New Test Accuracy: 91.18%, New Test Loss: 0.6987


In [ ]:
model.save("musician_recognition_model.h5")
print("✅ Model Saved Successfully!")

✅ Model Saved Successfully!


In [ ]:
model.save('/content/drive/MyDrive/musician_recognition_model.h5')